# Calibration/Prediction with TreeLSTM

Notice: we suppress training to only one epoch for conciseness of the script

In [1]:
# torch
import torch
from torch.utils.data import DataLoader

# in-house package
from Utils import *
from Models import *

In [2]:
# Global variables
BATCH_SIZE = 200
WORD_DIM = 200
BEST_VALID_LOSS = float('inf')
EMB_FILE = 'staticFile/glove.6B.' + str(WORD_DIM) + 'd.txt'
VOCAB_FILE = 'staticFile/vocab_kaggle.txt'
TRAIN_FILE = 'datasets/kaggle_train_tree.txt'
TEST_FILE = 'datasets/kaggle_test_tree.txt'
DEVICE = torch.device("cuda")

In [4]:
# 1) load data
trees, embeddings, vocab = text2DGL(TRAIN_FILE, VOCAB_FILE, EMB_FILE, WORD_DIM)
itos = dict()
for key in vocab.keys(): itos[vocab[key]] = key
total_trees = len(trees); ratio = 0.1
num_train = int(total_trees * (1. - ratio))
# training/dev spliting
train_loader = DataLoader(dataset = trees[:num_train], batch_size = BATCH_SIZE, collate_fn = batcher(DEVICE))
dev_loader = DataLoader(dataset = trees[num_train:], batch_size = BATCH_SIZE, collate_fn = batcher(DEVICE))

# Att'n Tree-LSTM (Pre-Training)

In [5]:
NUM_CLASS = 3; DROPOUT = 0.5
# create the model
ATT_TYPE = "ADD"
model = TreeLSTM(len(vocab), WORD_DIM, WORD_DIM, NUM_CLASS, DROPOUT, ATT_TYPE).to(DEVICE)
# use pretrained word emb
model.embedding.weight.data.copy_(torch.FloatTensor(embeddings).to(DEVICE))
model.embedding.weight.requires_grad = True
# create the optimizer
optimizer = torch.optim.Adam(model.parameters())

In [6]:
# 3) start training
NUM_EPOCH = 1
for i in range(NUM_EPOCH):
    loss, acc = training_treeLSTM(model, train_loader, optimizer, DEVICE, False)
    loss_v, acc_v = evaluation_treeLSTM(model, dev_loader, DEVICE)
    if loss_v < BEST_VALID_LOSS:
        BEST_VALID_LOSS = loss_v
        torch.save(model.state_dict(), 'text_tree_model_attn.pt')
        word_emb = saveWordVecFromModel(itos, model.embedding.weight.data)
    
    print("Epoch {:05d} | Train Loss {:.4f} | Train Acc {:.4f} |".format(i, loss, acc))
    print("Epoch {:05d} | Dev Loss {:.4f} | Dev Acc {:.4f} |".format(i, loss_v, acc_v))

Epoch 00000 | Train Loss 179.7654 | Train Acc 0.5771 |
Epoch 00000 | Dev Loss 141.6150 | Dev Acc 0.7124 |


# Tree-LSTM + Att'n + Enhanced (Pre-Training)

In [8]:
NUM_CLASS = 3; DROPOUT = 0.5
# create the model
ATT_TYPE = "ADD"
model = TreeLSTM(len(vocab), WORD_DIM, WORD_DIM, NUM_CLASS, DROPOUT, ATT_TYPE).to(DEVICE)
# use pretrained word emb
model.embedding.weight.data.copy_(torch.FloatTensor(embeddings).to(DEVICE))
model.embedding.weight.requires_grad = True
# create the optimizer
optimizer = torch.optim.Adam(model.parameters())

In [9]:
# 3) start training
NUM_EPOCH = 1
for i in range(NUM_EPOCH):
    loss, acc = training_treeLSTM(model, train_loader, optimizer, DEVICE, True)
    loss_v, acc_v = evaluation_treeLSTM(model, dev_loader, DEVICE, True)
    if loss_v < BEST_VALID_LOSS:
        BEST_VALID_LOSS = loss_v
        torch.save(model.state_dict(), 'text_tree_model_attn_plus.pt')
        word_emb = saveWordVecFromModel(itos, model.embedding.weight.data)
    
    print("Epoch {:05d} | Train Loss {:.4f} | Train Acc {:.4f} |".format(i, loss, acc))
    print("Epoch {:05d} | Dev Loss {:.4f} | Dev Acc {:.4f} |".format(i, loss_v, acc_v))

Epoch 00000 | Train Loss 461.7851 | Train Acc 0.5570 |
Epoch 00000 | Dev Loss 324.3966 | Dev Acc 0.6603 |


# Tree-LSTM + Att'n + Enhanced (Fine-Tuning)

In [11]:
VOCAB_FILE = 'staticFile/vocab_covid.txt'
TRAIN_FILE = 'datasets/covid_training_dgl.txt'
TEST_FILE = 'datasets/covid_test_dgl.txt'

In [12]:
# 1) load data
trees, embeddings, vocab_covid = text2DGL(TRAIN_FILE, VOCAB_FILE, EMB_FILE, WORD_DIM)
itos_covid = dict()
for key in vocab_covid.keys(): itos_covid[vocab_covid[key]] = key
total_trees = len(trees); ratio = 0.1
num_train = int(total_trees * (1. - ratio))
# training/dev spliting
train_loader = DataLoader(dataset = trees[:num_train], batch_size = BATCH_SIZE, collate_fn = batcher(DEVICE))
dev_loader = DataLoader(dataset = trees[num_train:], batch_size = BATCH_SIZE, collate_fn = batcher(DEVICE))

In [28]:
# load pre-trained word_embeddings
updated_wv = updateWordVecToModel(itos_covid, torch.Tensor(embeddings), word_emb, DEVICE)

In [32]:
NUM_CLASS = 3; DROPOUT = 0.5
# create the model
ATT_TYPE = "ADD"
model_covid = TreeLSTM(len(vocab_covid), WORD_DIM, WORD_DIM, NUM_CLASS, DROPOUT, ATT_TYPE).to(DEVICE)
# use pretrained word emb
model_covid.embedding.weight.data.copy_(torch.FloatTensor(updated_wv).to(DEVICE))
model_covid.embedding.weight.requires_grad = True
# create the optimizer
optimizer = torch.optim.Adam(model_covid.parameters())

In [36]:
# 3) start training
NUM_EPOCH = 1
BEST_VALID_LOSS = float('inf')
for i in range(NUM_EPOCH):
    loss, acc = training_treeLSTM(model_covid, train_loader, optimizer, DEVICE, True)
    loss_v, acc_v = evaluation_treeLSTM(model_covid, dev_loader, DEVICE, True)
    if acc_v < BEST_VALID_LOSS:
        BEST_VALID_LOSS = acc_v
        torch.save(model.state_dict(), 'covid_tree_attn_plus.pt')
    
    print("Epoch {:05d} | Train Loss {:.4f} | Train Acc {:.4f} |".format(i, loss, acc))
    print("Epoch {:05d} | Dev Loss {:.4f} | Dev Acc {:.4f} |".format(i, loss_v, acc_v))

Epoch 00000 | Train Loss 374.2436 | Train Acc 0.8061 |
Epoch 00000 | Dev Loss 309.3046 | Dev Acc 0.8122 |
